### SAのテンプレート
バッチサイズ分だけ並行して実施する

In [1]:
ex_num = "009"

In [2]:
import os
# 環境によって変更
# os.chdir('/home/jovyan/work/notebook')

In [3]:
class CFG:
    # path = "../model"    
    path = "../../output"
    base_file_path = "../input/sub20241221.csv"
    
    # 焼きなましのパラメータ
    temperature=100
    end_temperature=1.0
    cooling_rate=0.9999
    iteration=int(1e6)
    batch_size=32
    neighbor_types = ["swap", "insert", "inverse", "slide"]

In [4]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator
# 可視化用
import matplotlib.pyplot as plt

### データ読み込み

In [5]:
df = pd.read_csv(CFG.base_file_path)

In [6]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [7]:
df

,id,text
0,0,reindeer mistletoe elf gingerbread family adve...
1,1,reindeer sleep walk the night and drive mistle...
2,2,jingle yuletide carol cheer holly stocking chi...
3,3,sleigh the holly jingle unwrap gifts of magi r...
4,4,peppermint candy milk chocolate eggnog fruitca...
5,5,poinsettia yuletide eggnog fruitcake gingerbre...


### 関数

In [8]:
def plot_score_and_temperature(scores_history, temp_history):
    # x 軸: イテレーション
    x = range(len(scores_history))
    
    # Figure と Axes を作成
    fig, ax1 = plt.subplots(figsize=(8,5))
    
    # スコア用の軸 (左)
    color_score = 'tab:blue'
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Score (Perplexity)', color=color_score)
    ax1.plot(x, scores_history, label='Score', color=color_score)
    ax1.tick_params(axis='y', labelcolor=color_score)
    
    # 温度用の軸 (右) : twinx で第二のY軸を作成
    ax2 = ax1.twinx()
    color_temp = 'tab:red'
    ax2.set_ylabel('Temperature', color=color_temp)
    ax2.plot(x, temp_history, label='Temperature', color=color_temp)
    ax2.tick_params(axis='y', labelcolor=color_temp)
    
    # タイトル等
    plt.title('Score & Temperature Transition')
    fig.tight_layout()
    plt.show()

In [9]:
# 2単語組み合わせ全てから最低スコアとなるペアを探す
def find_initial_pair(
    CFG,
    scorer,
    words
):
    best_score = float('inf')
    best_pair = None
    for i in range(len(words)):
        perms = []
        for j in range(i+1, len(words)):
            pair = [words[i], words[j]]
            perms.append(" ".join(pair))
        
        scores = scorer.get_perplexity(perms, batch_size=CFG.batch_size)
        for j, perm in enumerate(perms):
            if scores[j] < best_score:
                best_score = scores[j]
                best_pair = perm.split(" ")
                
                # print(f"best_score: {best_score}, best_pair: {best_pair}")
     
    return best_pair, best_score

In [10]:
# 文字の順番をランダムにして、返す
def shuffle_initialize(
    scorer,
    words
):
    random.shuffle(words)
    return words, scorer.get_perplexity([" ".join(words)], batch_size=1)[0]

In [11]:
# 文字列のファイルからコストを最小化する文字列を生成する
def greedy_add_initialize(
    CFG,
    scorer,
    initial_words: list
):
    
    # 全単語
    candidates = initial_words[:]

    print("candidates:", candidates)
    print("-----初期配置の組み合わせ計算開始-----")
    # 初期配置を求める
    initial, initial_score = find_initial_pair(
        CFG,
        scorer,
        candidates
    )
    print("-----初期配置の組み合わせ計算終了-----")
    for w in initial:
        candidates.remove(w)

    print("initial:", initial, initial_score)
    current_order = initial[:]

    print("-----貪欲法での挿入開始-----")
    # 貪欲に残りを挿入
    while candidates:
        best_score = float('inf')
        best_word = None
        best_pos = None

        # 残っている単語それぞれについて
        for w in candidates:
            # current_orderのあらゆる挿入位置を試す
            # 挿入位置は0からlen(current_order)まで
            
            perms = []
            for pos in range(len(current_order) + 1):
                new_order = current_order[:pos] + [w] + current_order[pos:]
                perms.append(" ".join(new_order))
                
            scores = scorer.get_perplexity(perms, batch_size=CFG.BATCH_SIZE)
                
            for pos in range(len(current_order) + 1):
                if scores[pos] < best_score:
                    best_score = scores[pos]
                    best_word = w
                    best_pos = pos
                    
                    # print("best_word:", best_word, best_score)
        
        # best_wordをbest_posに挿入
        current_order.insert(best_pos, best_word)
        candidates.remove(best_word)
        print("current_order:", current_order, best_score)
    print("-----貪欲法での挿入終了-----")
    
    best_score = scorer.get_perplexity([" ".join(current_order)], batch_size=1)[0]
    
    return current_order, best_score

In [12]:
def generate_neighbor(sequence, neighbor_type='swap'):
    """
    sequence: list of words
    neighbor_type: str, 近傍生成の種類 ('swap', 'insert', etc.)
    """
    new_seq = sequence[:]
    length = len(new_seq)

    if neighbor_type == 'swap':
        # ランダムに2つの位置を選んで入れ替える
        a, b = random.sample(range(length), 2)
        new_seq[a], new_seq[b] = new_seq[b], new_seq[a]

    elif neighbor_type == 'insert':
        # ある単語を抜き出し、別の位置に挿入する
        idx = random.randint(0, length - 1)
        word = new_seq.pop(idx)
        new_pos = random.randint(0, length - 1)
        new_seq.insert(new_pos, word)

    elif neighbor_type == 'shuffle_segment':
        # ランダムな部分区間を決めてシャッフル
        start = random.randint(0, length - 2)
        end = random.randint(start + 1, length - 1)
        segment = new_seq[start:end+1]
        random.shuffle(segment)
        new_seq[start:end+1] = segment

    elif neighbor_type == 'reverse_segment':
        # ランダムな部分区間を決めて反転
        start = random.randint(0, length - 2)
        end = random.randint(start + 1, length - 1)
        segment = new_seq[start:end+1]
        new_seq[start:end+1] = list(reversed(segment))

    # 他にも様々な近傍操作を追加可能

    return new_seq

In [13]:
def search_annealing(
    scorer,
    initial_sequence,
    temperature: float = 1.0,
    end_temperature: float = 0.01,
    cooling_rate: float = 0.999,
    iteration: int = 1000,
    batch_size: int = 10,
    neighbor_types: list = None
):
    """
    焼きなまし法による探索

    Parameters
    ----------
    CFG: 設定用（必要に応じて）
    scorer: perplexityを計算できるオブジェクト
    initial_sequence: 初期の単語リスト
    temperature: 開始時の温度
    cooling_rate: 温度の減少率
    iteration: 探索の反復回数
    batch_size: 1ステップで生成する近傍候補の数
    neighbor_types: 近傍生成の種類を指定するリスト
    """
    if neighbor_types is None:
        neighbor_types = ['swap', 'insert', 'shuffle_segment', 'reverse_segment']

    # 初期状態
    current_seq = initial_sequence[:]
    best_seq = current_seq[:]
    best_score = scorer.get_perplexity([" ".join(current_seq)], batch_size=1)[0]

    # current_scoreの計算
    current_score = best_score

    # スコア推移を記録するリスト
    scores_history = []
    temp_history = []
    
    # 現在の状態を持つリスト
    current_seqs = []
    current_scores = []
    
    # 初期解の生成
    for _ in range(CFG.batch_size):
        print("current_seq:", current_seq)
        print("current_score:", current_score)
        current_seq, current_score = shuffle_initialize(
            scorer,
            current_seq,
        )
        current_seqs.append(current_seq)
        current_scores.append(current_score)
        
    for itr in tqdm(range(iteration)):
        # 記録（ループの冒頭 or 最後のどちらでもOK）
        scores_history.append(current_scores[0])
        temp_history.append(temperature)
        
        # 近傍候補をまとめて生成
        neighbor_candidates = []
        for i in range(CFG.batch_size):
            # 近傍操作の種類をランダムに選ぶ
            n_type = random.choice(neighbor_types)
            neighbor_seq = generate_neighbor(current_seqs[i], n_type)
            neighbor_candidates.append(neighbor_seq)

        
        # まとめてスコアを計算（GPUを活用）
        texts = [" ".join(seq) for seq in neighbor_candidates]
        # print("texts:", texts)
        
        scores = scorer.get_perplexity(texts, batch_size=batch_size)
        # print("scores:", scores)
        
        # 候補の中から1つを焼きなまし法の確率的受容ルールで選ぶ
        for i, (cand_seq, cand_score) in enumerate(zip(neighbor_candidates, scores)):
            delta = cand_score - current_scores[i]
            accept_prob = math.exp(-delta / temperature) if delta > 0 else 1.0
                
            # スコアが良ければ必ず受容
            if (itr==0) or (random.random() < accept_prob):
                current_scores[i] = cand_score
                current_seqs[i] = cand_seq

        # ベストの更新
        # currnet_scoresのリストの最小値を取得
        if np.min(current_scores) < best_score:
            # current_scoresの最小値のインデックスを取得
            idx = np.argmin(current_scores)
            
            best_score = current_scores[idx]
            best_seq = current_seqs[idx]
            print(f"iter {itr}: best_score: {best_score}")
            print("best_seq:", best_seq)

        # 温度を下げる
        if itr % 10 == 0:
            temperature *= cooling_rate
        
        if itr % 1000 == 0:
            print(f"iter :{itr}, best_score: {best_score}, temperature: {temperature}")

            # current_seqsを結合して文字列保存
            current_seqs_text = [" ".join(seq) for seq in current_seqs]
            # current_seqsとcurrent_scoresのリストをdfとして保存
            df = pd.DataFrame(current_seqs_text)
            df["score"] = current_scores
            
            # indexとなっていたものをsequencre列に変更
            df = df.reset_index()
            df = df.rename(columns={"index": "sequence"})
            
            df.to_csv(f"current_texts.csv", index=False)
        if temperature < end_temperature:
            break
        
    return best_seq, best_score, scores_history, temp_history

In [14]:
def search_optimal_permutation(
    CFG,
    id: int, 
    scorer
):
    
    df = pd.read_csv(CFG.base_file_path)
    
    start = time.time()
    # 文字列を単語に分割
    words = df.loc[id,"text"].split()
    
    initial_sequence = words[:]
    print("--------初期配置の組み合わせ計算開始---------")
    initial_sequence, initial_score = shuffle_initialize(
        scorer=scorer,
        words=words
    )
    
    print(f"{id}th sample: {initial_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"initial_score: {initial_score}")
    print("--------初期配置の組み合わせ計算終了---------")
    
    print("---------焼きなまし処理開始---------")
    best_sequence, best_score, scores_history, temp_history = search_annealing(
        scorer=scorer,
        initial_sequence=initial_sequence,
        temperature=CFG.temperature,
        end_temperature=CFG.end_temperature,
        cooling_rate=CFG.cooling_rate,
        iteration=CFG.iteration,
        batch_size=CFG.batch_size,
        neighbor_types=CFG.neighbor_types
    )
    print("---------焼きなまし処理終了---------")
    
    print(f"{id}th sample: {best_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"best_score: {best_score}")
    
    plot_score_and_temperature(scores_history, temp_history)
    
    return best_sequence, best_score


In [15]:
# 文字列型にのみダブルクォーテーションを付ける関数
def add_quotes_to_strings(value):
    if isinstance(value, str):  # 文字列型の場合
        return f'"{value}"'
    return value  # それ以外はそのまま

In [16]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{CFG.path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [17]:
# scorer.clear_gpu_memory()

In [ ]:
for i in [4]:
    
    print(f"Processing {i}th sample")
    best_sequence, best_score = search_optimal_permutation(
        CFG,
        id=i,
        scorer=scorer,
    )
    
    print("Best sequence:", best_sequence)
    print("Best score:", best_score)
    sub_df.loc[i, "best_value"] = best_score
    sub_df.loc[i, "text"] = " ".join(best_sequence)



Processing 4th sample
--------初期配置の組み合わせ計算開始---------
4th sample: ['kaggle', 'of', 'paper', 'hohoho', 'peppermint', 'card', 'toy', 'eggnog', 'season', 'merry', 'snowglobe', 'wrapping', 'that', 'from', 'wonder', 'we', 'as', 'night', 'cookie', 'hope', 'and', 'star', 'puzzle', 'dream', 'wreath', 'game', 'not', 'chocolate', 'to', 'the', 'fireplace', 'have', 'angel', 'candy', 'peace', 'bow', 'it', 'fruitcake', 'candle', 'believe', 'joy', 'poinsettia', 'milk', 'wish', 'workshop', 'with', 'greeting', 'you', 'doll', 'in']
Elapsed time: 0.76 sec
initial_score: 1745.582779603376
--------初期配置の組み合わせ計算終了---------
---------焼きなまし処理開始---------
current_seq: ['kaggle', 'of', 'paper', 'hohoho', 'peppermint', 'card', 'toy', 'eggnog', 'season', 'merry', 'snowglobe', 'wrapping', 'that', 'from', 'wonder', 'we', 'as', 'night', 'cookie', 'hope', 'and', 'star', 'puzzle', 'dream', 'wreath', 'game', 'not', 'chocolate', 'to', 'the', 'fireplace', 'have', 'angel', 'candy', 'peace', 'bow', 'it', 'fruitcake', 'candle'

  0%|          | 0/1000000 [00:00<?, ?it/s]

iter 0: best_score: 1262.217028265557
best_seq: ['believe', 'card', 'paper', 'merry', 'that', 'fruitcake', 'joy', 'toy', 'hohoho', 'bow', 'it', 'doll', 'wrapping', 'as', 'poinsettia', 'have', 'chocolate', 'we', 'in', 'of', 'you', 'star', 'game', 'wish', 'candy', 'wonder', 'cookie', 'to', 'greeting', 'peppermint', 'puzzle', 'snowglobe', 'fireplace', 'milk', 'the', 'not', 'candle', 'night', 'with', 'hope', 'from', 'season', 'and', 'peace', 'eggnog', 'kaggle', 'wreath', 'workshop', 'angel', 'dream']
iter :0, best_score: 1262.217028265557, temperature: 99.99
iter 2: best_score: 1135.8723783859068
best_seq: ['peppermint', 'paper', 'merry', 'that', 'fruitcake', 'joy', 'toy', 'hohoho', 'bow', 'it', 'doll', 'wrapping', 'as', 'poinsettia', 'have', 'chocolate', 'we', 'in', 'of', 'you', 'star', 'game', 'wish', 'candy', 'wonder', 'cookie', 'to', 'greeting', 'believe', 'puzzle', 'snowglobe', 'fireplace', 'milk', 'the', 'card', 'not', 'candle', 'night', 'with', 'hope', 'from', 'season', 'and', 'peac

In [ ]:
sub_df

In [ ]:
# 各セルに関数を適用
sub_df["text"] = sub_df["text"].astype(str)
sub_df = sub_df.applymap(add_quotes_to_strings)

In [ ]:
sub_df[["id", "text"]].to_csv(f"{CFG.path}/out/submission_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)
sub_df.to_csv(f"{CFG.path}/out/score_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)

In [ ]:
np.mean(sub_df["best_value"])